In [ ]:
#!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 5.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import emoji

In [ ]:
current_language='-' #Set to current language
path = '/' #Change to folder path


#File paths: Adjust to relevant paths
file1 = '/en/refugee_OR_refugees_OR_refugees_OR_refugees_OR_Ref - Feb 27, 2024 - 3 27 39 PM.xlsx'
file2 = '/en/refugee_OR_refugees_OR_refugees_OR_refugees_OR_Ref - Feb 27, 2024 - 3 30 39 PM.xlsx'
file3 = '/en/refugee_OR_refugees_OR_refugees_OR_refugees_OR_Ref - Feb 27, 2024 - 3 35 31 PM.xlsx'
file4 = '/en/refugee_OR_refugees_OR_refugees_OR_refugees_OR_Ref - Feb 27, 2024 - 3 39 03 PM.xlsx'
file5 = '/en/refugee_OR_refugees_OR_refugees_OR_refugees_OR_Ref - Feb 27, 2024 - 3 43 56 PM.xlsx'


data = [file1, file2, file3, file4, file5]

df = pd.concat((pd.read_excel(path+file) for file in data), ignore_index=True)

# Clean data

In [ ]:
#FILTER
# Keep rows where language is the current language
df = df[df['Language'] == current_language]

# Keep rows where source is 'Twitter'
df = df[df['Source'] == 'Twitter']

In [ ]:
#DUPLICATES
# Remove duplicate rows
df = df.drop_duplicates()

# Remove url duplicates
df = df.drop_duplicates(subset=['Hit Sentence'])

# Drop rows with NaN values in 'Hit Sentence' column from the original DataFrame
df = df.dropna(subset=['Hit Sentence'])


In [ ]:
# Columns to drop
columns_to_drop = [
    'Social Echo Total', 'Editorial Echo', 'Twitter Social Echo', 'Facebook Social Echo', 'Reddit Social Echo',
    'Views', 'Likes', 'Replies', 'Retweets', 'Comments', 'Shares', 'Reactions', 'Engagement', 'Estimated Views',
    'Desktop Reach', 'Mobile Reach', 'National Viewership', 'AVE', 'Reach', 'Headline', 'Opening Text', 'Source', 'Document Tags',
    'Twitter Authority', 'Twitter Followers', 'Twitter Following', 'Twitter Id', 'Twitter Client', 'Twitter Screen Name', 'Twitter Bio',
    'Country', 'State', 'City', 'Subregion', 'Influencer', 'User Profile Url', 'Threads', 'Is Verified',
    'Tweet Id', 'Parent URL', 'Key Phrases', 'Document ID',
    'Input Name', 'Keywords', 'Alternate Date Format', 'Time'
]

# Create a clean DataFrame by dropping specified columns
df_clean = df.drop(columns=columns_to_drop)


# Text processing - mentions, emojis and URLs

In [ ]:
df_new =  df_clean[:]

# Replace mentions starting with '@' with '@USER'
df_new['Hit Sentence'] = df_new['Hit Sentence'].str.replace(r'@\w+', '@USER')


In [ ]:
# Function to transform emojis into text
def transform_emojis(text):
    return emoji.demojize(text)

# Apply the function to the 'Hit Sentence' column
df_new['Hit Sentence'] = df_new['Hit Sentence'].apply(transform_emojis)


In [ ]:
# Define a regex pattern to match URLs
url_pattern = r'https?://\S+|www\.\S+'

# Function to replace URLs with 'URL'
def replace_urls_with_url(text):
    return re.sub(url_pattern, 'URL', text)

# Apply the function
df_new['Hit Sentence'] = df_new['Hit Sentence'].apply(replace_urls_with_url)

# Save cleaned datasets

Create a Sample dataset for the labelling process

In [ ]:
# Shuffle the rows in df_test
df_shuffled = df_new.sample(frac=1, random_state=42)

# Reset the index of the shuffled DataFrame
df_shuffled.reset_index(drop=True, inplace=True)
df_shuffled.head()

# Sample 400 random rows per languege
df_sampled = df_shuffled.sample(n=400, random_state=42)
df_sampled.info()

In [ ]:
# Save the sampled DataFrame to a CSV file
df_sampled.to_csv('/path/df.csv', sep='\t', encoding='utf-16', index=False)

Saving the non-sampled tweets to another file:

In [ ]:
df_new = df_new[~df_new['URL'].isin(df_sampled['URL'])]

# Save the DataFrame to a CSV file in Google Drive
df_new.to_csv('/path/df.csv', sep='\t', encoding='utf-16', index=False)